In [6]:
class Config:
    def __init__(self, config_path):
        self.config_path = config_path
        with open(self.config_path, 'r') as f:
            self.dict = json.load(f)
            
        assert 'bot_token' in self.dict
        assert 'admin_password' in self.dict
        
        if 'enabled_group_ids' not in self.dict:
            self.dict['enabled_group_ids'] = []

    def save_to_disk(self):
        with open(self.config_path, 'w') as f:
            json.dump(self.dict, f)

    @property
    def bot_token(self):
        return self.dict['bot_token']

    @property
    def admin_password(self):
        return self.dict['admin_password']

    @property
    def enabled_group_ids(self):
        list_ids = [] if 'enabled_group_ids' not in self.dict else self.dict['enabled_group_ids']
        return tuple(list_ids)

    @enabled_group_ids.setter
    def enabled_group_ids(self, value):
        self.dict['enabled_group_ids'] = list(value)
        self.save_to_disk()

    def add_enabled_group_id(self, group_id):
        list_ids = list(self.enabled_group_ids)
        assert group_id not in list_ids
        list_ids.append(group_id)
        self.enabled_group_ids = list_ids

    def remove_enabled_group_id(self, group_id):
        list_ids = list(self.enabled_group_ids)
        assert group_id in list_ids
        list_ids.remove(group_id)
        self.enabled_group_ids = list_ids


In [7]:
from pathlib import Path
import telebot
import logging
from logging.handlers import TimedRotatingFileHandler
import json

LOG_FORMAT = '%(module)-12s: %(asctime)s %(levelname)s %(message)s'

default_logger = logging.getLogger('default_logger')
default_logger.setLevel(logging.DEBUG)
default_log_formatter = logging.Formatter(LOG_FORMAT)

app_dir = Path('..')

log_dir = app_dir/'logs'
log_dir.mkdir(exist_ok=True, parents=True)
log_file_handler = TimedRotatingFileHandler(log_dir/"log.log",
                                            when="midnight",
                                            backupCount=30)
log_file_handler.setFormatter(default_log_formatter)
log_file_handler.setLevel(logging.INFO)
default_logger.addHandler(log_file_handler)


log_console_handler = logging.StreamHandler()
# log_console_handler.setLevel(logging.INFO)
log_console_handler.setLevel(logging.DEBUG)
log_console_handler.setFormatter(default_log_formatter)
default_logger.addHandler(log_console_handler)

logger = default_logger

data_dir = app_dir/'data'
config = Config(data_dir/'config.json')
# Commenter dev
# bot_token = '1403177107:AAErtBuBloLUb3U4zduNGpEil_p8wOh3NdI'

# data_dir = Path('data')
# with open(data_dir/'config.json') as f:
#     config = json.load(f)
# assert 'bot_token' in config
# assert 'admin_password' in config
    
# logger.info('abc')

In [8]:
import argparse
import shlex

class CommandParser(argparse.ArgumentParser):
    def __init__(self):
        super().__init__(add_help=False, formatter_class=argparse.RawTextHelpFormatter)
        self.setup_rules()

    def setup_rules(self):
        command_options = {
            "enable": "Enables the bot for a given group",
            "disable": "Disables the bot for a given group",
            "help": "Print help for the commands",
        }

        self.add_argument('command',
                          choices=command_options.keys(),
                          type = str.lower)

        self.add_argument('--password',
                          type=str,
                          metavar='',
                          help=f'Password to access the bot')
        self.add_argument('--group_id',
                          type=int,
                          metavar='',
                          help='Group id to perform operation on')

        self.usage = f'[command] [parameters]'

    def validate_and_fix_params(self, params):
        pass # Empty for now

    @classmethod
    def unify_whitespace(cls, s):
        # Ws can be obtained using the following code
        #       all_unicode = ''.join(chr(c) for c in range(sys.maxunicode+1))
        #       ws = ''.join(re.findall(r'\s', all_unicode))
        ws = '\t\n\x0b\x0c\r\x1c\x1d\x1e\x1f \x85\xa0\u1680\u2000\u2001\u2002\u2003\u2004\u2005\u2006\u2007\u2008\u2009\u200a\u2028\u2029\u202f\u205f\u3000'
        unified_ws = ' '
        s2 = s.translate(s.maketrans(ws, unified_ws * len(ws)))
        return s2


    def parse_command(self, command):
        # We unify whitespace as shlex can't parse Unicode whitespaces properly
        command = self.unify_whitespace(command)
        # Fix back Telegram autoreplace
        command = command.replace('—', '--')
        args = shlex.split(command)

        return self.parse_args(args)

    def parse_args(self, *args, **kwargs):
        self.error_message = None
        try:
            params = super().parse_args(*args, **kwargs)
            # Make checks for command mandatory params to be set
            self.validate_and_fix_params(params)
            return params
        except Exception as e:
            self.error_message = str(e)

    def error(self, message):
        """
        Overwrite the default parser behaviour in case of error - throws an exception instead of making exit
        """
        message = f'{message}\n*use "help" command to get help*'
        raise Exception(message)

    def format_help(self):
        help = super().format_help()
        # default "optional arguments" name isn't suitable for Toffee as some of these arguments are mandatory for some commands
        help = help.replace("optional arguments", "parameters")
        # in our case "positional arguments" are used to set the command
        help = help.replace("positional arguments", "command")
        return help

In [9]:
# args = command_parser.parse_command('enable --group_id -12312312312')
# print(command_parser.error_message)
# args

In [10]:
bot = telebot.TeleBot(config.bot_token, parse_mode='HTML') # You can set parse_mode by default. HTML or MARKDOWN

command_parser = CommandParser()

@bot.message_handler(commands=['start', 'help'])
def send_welcome(message):
    try:
        help_message = \
"""
I'm Anonymizer bot. You can use me to establish anonymous comments in your channel.
If you want to use me in your channel do the following:
1️⃣ Go to @anocom_chat, provide the name of the discussion group of your channel and ask to enable anocom_bot for it.
2️⃣ Wait for my admin to confirm your request.
3️⃣ Add me to your channel's discussion group and write at least one message in this group.
4️⃣ Message @anocom_chat about step 3️⃣ completion.
5️⃣ Provide me with an admin rights to "Delete messages" and "Ban users" in your discussion group.

❗️Note that after you enable Anonymizer in your group all the messages in the group will be anonymized and users will be kicked from the group after each message.
"""
        bot.reply_to(message, help_message)
    except Exception as e:
        logger.exception(e)
    
@bot.message_handler(func=lambda m: True, content_types=['new_chat_members', 'left_chat_member'])
def handle_member_join_leave(message):
    try:
        bot.delete_message(message.chat.id, message.message_id)
    except Exception as e:
        logger.exception(e)

    
@bot.message_handler(func=lambda m: True, content_types=['text', 'audio', 'document', 'photo', 'sticker', 'video', 'video_note', 'voice', 'location', 'contact'])
def handle_message(message):
    try:
        logger.debug(message)

        try_handle_commands(message)
        try_resend_message(message)
    except Exception as e:
        logger.exception(e)

def try_handle_commands(message):
    if message.chat.type != 'private':
        logger.debug('Message ignored by command handler as it was sent not in private chat')
        return
    
    if message.content_type != 'text':
        logger.debug('Message ignored by command handler as it is not a text message')
        return
    
    text = message.text
    if text is None:
        logger.debug('Message ignored by command handler as it does not have text')
        return
    
    
#     args = command_parser.parse_command('enable --group_id -12312312312 --password DJ6{eqrg%Jmh6B7h')
    args = command_parser.parse_command(text)
    
    if command_parser.error_message is not None:
        logger.debug(f'Message ignored by command handler as parser raised an error "{command_parser.error_message}"')
        return # Most likely it's not a command, just ignore it
    
    logger.info(f'Bot command: "{args.command}"')
    if args.password is None:
        logger.warning('Command requested without a password')
        return
        
    if args.password != config.admin_password:
        logger.warning('Command requested with a wrong password')
        return
        
    if args.command == 'help':
        help_message = '<pre>' + command_parser.format_help() + '</pre>'
        bot.send_message(message.chat.id, help_message)
        return
        
    if args.command in ('enable', 'disable'):
        group_id = args.group_id
        if group_id is None:
            logger.info(f'Received {args.command} command without group_id')
            bot.send_message(message.chat.id, '--group_id is required')
            return
        
        if args.command == 'enable':
            if group_id in config.enabled_group_ids:
                logger.info(f'Group {group_id} is already enabled')
                bot.send_message(message.chat.id, f'Group {group_id} is already enabled')
                return
            config.add_enabled_group_id(group_id)
            logger.info(f'Group {group_id} enabled')
            bot.send_message(message.chat.id, f'Group {group_id} enabled')
            return
                
        elif args.command == 'disable':
            if group_id not in config.enabled_group_ids:
                logger.info(f'Group {group_id} is not enabled')
                bot.send_message(message.chat.id, f'Group {group_id} is not enabled')
                return
            config.remove_enabled_group_id(group_id)
            logger.info(f'Group {group_id} disabled')
            bot.send_message(message.chat.id, f'Group {group_id} disabled')
            return
        else:
            raise Exception(f'Unhandled command {args.command}')
        

def try_resend_message(message):
    
    if message.chat.type not in ("group", "supergroup"):
        return
    
    if message.chat.id not in config.enabled_group_ids:
        logger.info(f'Ignoring message in disabled chat {message.chat.id} "{message.chat.title}"')
        return

    # telegram_notifications_user_id makes messages forwarding from the channel to the group
    telegram_notifications_user_id = 777000
    if message.from_user.id == telegram_notifications_user_id:
        logger.debug('Skipping telegram_notifications_user_id message')
        return

    logger.info(f'Anonymizing message, chat {message.chat.id} "{message.chat.title}"')

    # Delete the message
    try:
        bot.delete_message(message.chat.id, message.message_id)
    except Exception as e:
        error_message = '🚫Cannot anonymize the message. Give me "Delete messages" admin right.'
        bot.send_message(message.chat.id, error_message)
        logger.warning(f'Cannot delete message, no admin rights, chat {message.chat.id} "{message.chat.title}"')
        return

    # Remove the user from the group
    is_admin = False
    try:
        member = bot.get_chat_member(chat_id=message.chat.id, user_id=message.from_user.id)
        logger.debug(member)
        # group_anonymous_bot_id - bot that makes group admins anonymous
        group_anonymous_bot_id = 1087968824
        is_admin = (member.status in ('creator', 'administrator')) \
            or (message.from_user.id == group_anonymous_bot_id)
        if not is_admin:
            try:
                bot.kick_chat_member(chat_id=message.chat.id, user_id=message.from_user.id)
                bot.unban_chat_member(chat_id=message.chat.id, user_id=message.from_user.id)
            except Exception as e:
                error_message = '🚫Cannot remove the user from the chat to ensure anonymity. Give me "Ban users" admin right.'
                bot.send_message(message.chat.id, error_message)
                logger.warning(f'Cannot delete user, no admin rights, chat {message.chat.id} "{message.chat.title}"')

    except Exception as e:
        logger.exception(e)

    # Don't resend message if it contains links for non admins
    should_resend = True
    if not is_admin:
        entities = []
        if 'entities' in message.json:
            entities.extend(message.json['entities'])
        if 'caption_entities' in message.json:
            entities.extend(message.json['caption_entities'])
        unallowed_entity_types = ['mention', 'url', 'text_link', 'email', 'phone_number']
        contains_unallowed_entity = any([x['type'] in unallowed_entity_types for x in entities])
        should_resend = should_resend and (not contains_unallowed_entity)

    # Resend message 
    if should_resend:
        reply_to_message_id = None if (message.reply_to_message is None) else message.reply_to_message.message_id

        if message.content_type == 'text':
            if message.reply_to_message is None:
                bot.send_message(message.chat.id, message.html_text)
            else:
                bot.reply_to(message.reply_to_message, message.html_text)
        elif message.content_type == 'photo':
            photos = message.json['photo']
            if len(photos) >= 1:
                bot.send_photo(message.chat.id,
                               photos[0]['file_id'],
                               caption=message.html_caption,
                               reply_to_message_id=reply_to_message_id)
            else:
                print("Strange photo with 0 photos", message)
        elif message.content_type == 'sticker':
            bot.send_sticker(message.chat.id,
                             message.sticker.file_id,
                             reply_to_message_id=reply_to_message_id)
        elif message.content_type == 'document':
            bot.send_document(message.chat.id,
                              message.document.file_id,
                              caption=message.html_caption,
                              reply_to_message_id=reply_to_message_id)
        elif message.content_type == 'audio':
            bot.send_audio(message.chat.id,
                           message.audio.file_id,
                           caption=message.html_caption,
                           reply_to_message_id=reply_to_message_id)
        elif message.content_type == 'video':
            bot.send_video(message.chat.id,
                           message.video.file_id,
                           caption=message.html_caption,
                           reply_to_message_id=reply_to_message_id)
        elif message.content_type in ('video_note', 'voice', 'location', 'contact'):
            # These are ignored as potentially de-anonymising messages
            pass
        else:
            logger.error(f'Unhandled content_type: {message}')
    else:
        logger.debug('Skipping message resend as it contains unallowed entities')
        

In [ ]:
bot.polling()

In [18]:
1

1

In [11]:
# -1001405427290 - channel
# -1001170882970 - group

bot.send_message(-1001405427290, 'Пост от бота')

In [12]:
from telebot import types

In [33]:
markup = types.InlineKeyboardMarkup()

# markup.add(types.InlineKeyboardButton(text='abc',
#                                       callback_data="['value', '" + 'abc' + "', '" + 'def' + "']"),
#            types.InlineKeyboardButton(text='X',
#                                       callback_data="['key', '" + 'def' + "']"))
text = \
"""
В воскресенье нужно собираться в колонны на своих районах и от туда уже идти в центр. К примеру: 12.00 сбор на районе(выбрать большой перекресток или площадь) без опоздания. И в 12.10 выходить в к центру что бы к 12.30 всем колонам быть в центре. Это если не поменяется схема на воскресный марш.
"""
text = 'Нравится идея?'
markup.add(types.InlineKeyboardButton(text='👍(1)', url='google.com'))
bot.send_message(-1001405427290, text, reply_markup=markup)